In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt

In [2]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)
# np.savez('fishing.npz', X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
#https://github.com/ioskn/mldl_htwg/raw/master/data/fishing.npz
import os
from urllib.request import urlretrieve
if not os.path.isfile('fishing.npz'):
    print("Downloading")
    urlretrieve('http://www-home.htwg-konstanz.de/~oduerr/data/fishing.npz',filename = 'fishing.npz')
d = np.load('fishing.npz')
X_train = d['Xt'] #"livebait","camper","persons","child"
X_test = d['Xte']
y_train = d['yt']
y_test = d['yte']
pd.DataFrame(X_train[0:2])

,0,1,2,3
0,1.0,0.0,4.0,0.0
1,1.0,1.0,2.0,0.0


In [3]:
tfd = tfp.distributions
tfb = tfp.bijectors

np.random.seed(42)
tf.random.set_seed(42)

inputs = Input(shape=(X_train.shape[1],))
rate = Dense(1, activation=tf.exp)(inputs) # Definition of a single layer with one output
p_y = tfp.layers.DistributionLambda(tfd.Poisson)(rate) # We use exponential of the output to model the rate

# Glueing the NN and the output layer together. Note that output p_y is a tf.distribution
model_p = Model(inputs=inputs, outputs=p_y)

# The second argument is the output of the model and thus a tfp-distribution. It's as simple as calling log_prob to calculate the log-probability of the observation that is needed to calculate the NLL.
def NLL(y_true, y_hat):
  return -y_hat.log_prob(y_true)

model_p.compile(Adam(learning_rate=0.01), loss=NLL)
model_p.summary()

Instructions for updating:
The `interpolate_nondiscrete` flag is deprecated; instead use `force_probs_to_zero_outside_support` (with the opposite sense).
Instructions for updating:
The `interpolate_nondiscrete` property is deprecated; instead use `force_probs_to_zero_outside_support` (with the opposite sense).
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
 distribution_lambda (Distri  ((None, 1),              0         
 butionLambda)                (None, 1))                         
                                                                 
Total params: 5
Trainable params: 5
Non-trainable params: 0
___________________

In [5]:
history = model_p.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), epochs=2000)

7/7 [==============================] - 0s 5ms/step - loss: 3.5710 - val_loss: 3.2810
Epoch 187/2000
7/7 [==============================] - 0s 5ms/step - loss: 3.5683 - val_loss: 3.2789
Epoch 188/2000
7/7 [==============================] - 0s 5ms/step - loss: 3.5661 - val_loss: 3.2750
Epoch 189/2000
7/7 [==============================] - 0s 5ms/step - loss: 3.5652 - val_loss: 3.2718
Epoch 190/2000
7/7 [==============================] - 0s 5ms/step - loss: 3.5592 - val_loss: 3.2697
Epoch 191/2000
7/7 [==============================] - 0s 5ms/step - loss: 3.5667 - val_loss: 3.2701
Epoch 192/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.5614 - val_loss: 3.2662
Epoch 193/2000
7/7 [==============================] - 0s 5ms/step - loss: 3.5598 - val_loss: 3.2628
Epoch 194/2000
7/7 [==============================] - 0s 5ms/step - loss: 3.5557 - val_loss: 3.2586
Epoch 195/2000
7/7 [==============================] - 0s 5ms/step - loss: 3.5522 - val_loss: 3.2531
Epoch 196/2000


In [ ]:
history = model_p.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), epochs=2000)
plt.style.use('default')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(y_test)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
model = Model(inputs=inputs, outputs=p_y)
y_hat_test = model.predict(X_test).flatten()


rmse=np.sqrt(np.mean((y_test - y_hat_test)**2))
mae=np.mean(np.abs(y_test - y_hat_test))

NLL = model_p.evaluate(X_test, y_test) #returns the NLL

df3 = pd.DataFrame(
         { 'RMSE' : rmse, 'MAE' : mae, 'NLL' : NLL}, index=['Poisson Regression (TFP)']
)
pd.concat([df2,df3])

In [ ]:
from scipy.stats import poisson
lower=poisson.ppf(0.025, y_hat_test)
upper=poisson.ppf(0.975, y_hat_test)

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.scatter(y_hat_test, y_test, alpha=0.3)
plt.title('Comparison on the testset')
plt.xlabel('predicted average of caught fish')
plt.ylabel('observed number of caught fish')
plt.plot(y_hat_test[np.argsort(y_hat_test,axis=0)].flatten(), lower[np.argsort(y_hat_test,axis=0)],linestyle='dashed',c="black")
plt.plot(y_hat_test[np.argsort(y_hat_test,axis=0)].flatten(), upper[np.argsort(y_hat_test,axis=0)],linestyle='dashed',c="black")
plt.plot(y_hat_test, y_hat_test, c="black")